In [1]:
import tensorflow as tf
import tensorflow_hub as hub

/home/nab/work/DeepPavlov/venv-dp/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_datapath = 'train_poor.csv'  # пока что подразумеваем, что загрузка уже произведена

Ранее мы использовали стандартные input_fn из tf.estimator, чтобы кормить данные MyEstimator, однако далеко не всегда удобно грузить весь датасет в DataFrame pandas или в массивы numpy, которые полностью живут в оперативной памяти, поэтому в tf возможно вместо high-level Estimator API использовать mid-level Dataset API.

In [4]:
snips_ds = tf.data.TextLineDataset(train_datapath).skip(1)  # сразу же пропускаем первую строчку с заголовками
snips_ds

<SkipDataset shapes: (), types: tf.string>

Теперь нужно преобразовать каждую строку (каждый элемент выборки) нашего датасета, чтобы уточнить, где в этой строке можно найти фичи, а где разметку для обучения, и как всё это преобразовывать перед подачей на вход модели. Для выполнения такого преобразования нашего snips_ds, как и у любого tf.data.Dataset, есть метод map, принимающий на вход функцию, которую опишем ниже.

In [23]:
def _parse_line(line: tf.Tensor): # на входе тензор типа tf.string и это предвещает большие проблемы...
    
    text_string, label_string = tf.decode_csv(records=line, record_defaults=[[""], [""]])  # есть даже спец. функция
    # есть ещё варианты парсить tf.string в tf.train.Example, если кто-то им пользовался
    label_string = tf.string_to_hash_bucket_fast(input=label_string, num_buckets=7)  # на этот раз без словаря для лейблов
    features = {'text': text_string}  # MyEstimator с помощью ключей словаря матчит данные с feature_column
    labels = label_string  # разметку тоже можно паковать в словарь, если MyEstimator поддерживает это
    
    return features, labels # train_input_fn должна возвращать tf.data.Dataset, состоящий из таких вот кортежей

В принципе можно с помощью tf.py_func() как-то возможно более просто делать преобразования, однако не все такие преобразования будут выполняться на плюсовом бекэнде tf, из-за чего может возникнуть ботлнек, заставляющий GPU простаивать и как следствие замедляющий время обучения.

In [12]:
snips_ds = snips_ds.map(_parse_line) # map применяет к каждому элементу переданную функцию и возвращает новый объект
snips_ds

<MapDataset shapes: ({text: ()}, ()), types: ({text: tf.string}, tf.int64)>

Мы объявили, как будут считаны и обработаны данные, но само считывание будет происходить в момент обучения, чтобы поменять требования по оперативной памяти (датасет туда может не влезать) на требования к вычислительным ресурсам (считывание данных делает процессор, и если операции матричного перемножения выполняются на GPU/TPU, то процессор всё равно не так уж и загружен). Объявим ещё и то, как данные будут подготовлены для обучения MyEstimator.

In [13]:
snips_ds = snips_ds.repeat(1050) # если repeat не передать аргументов, то только MyEstimator сам сможет остановить обучение
snips_ds

<RepeatDataset shapes: ({text: ()}, ()), types: ({text: tf.string}, tf.int64)>

Каждый элемент датасета тоже по сути маленький датасет, но есть и отличия в методах, которые как-то явно в паре слов не описать. На примере батчевания можно сказать, что hub.text_embedding_column ждёт только батчей, поэтому как минимум .batch(1) к датасету нужно применить.

In [25]:
snips_ds = snips_ds.batch(4) # здесь нужно быть внимательным, потому что это влияет на размерность тензоров, и даже
snips_ds

<BatchDataset shapes: ({text: (?,)}, (?,)), types: ({text: tf.string}, tf.int64)>

Есть ещё несколько методов:
.apply() (похож на .map(), но может делать преобразования всего датасета, а не поэлементно);
.shuffle() (понятно, что делает);
однако непонятно, какие есть ограничения на порядок вызова этих методов на датасете.

Попробуем скормить наш датасет MyEstimator, на этот раз используем болванку - BaselineClassifier - который вообще не использует фичи, а только лейблы.

In [26]:
my_estimator = tf.estimator.BaselineClassifier(n_classes=7)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpnz8nv0gg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdef87f1860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


train_input_fn, которая передаётся в MyEstimator.train() должна возвращать просто объект класса tf.data.Dataset. Нужно именно писать отдельную функцию, а не передавать заранее сконструированный датасет через лямбда-функцию; связано это с процедурой построение графа для MyEstimator.

input_fn не должна принимать аргументов, но если очень захотеть, то можно использовать ту же лямбду.

In [27]:
def snips_train_input_fn():
    snips_ds = tf.data.TextLineDataset(train_datapath).skip(1)
    snips_ds = snips_ds.map(_parse_line)
    snips_ds = snips_ds.batch(32)
    return snips_ds
my_estimator.train(input_fn=snips_train_input_fn, max_steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpnz8nv0gg/model.ckpt.
INFO:tensorflow:loss = 13.621372, step = 1
INFO:tensorflow:Loss for final step: 13.621372.


In [30]:
eval_info = my_estimator.evaluate(input_fn=snips_train_input_fn) # снова скорим на обучающей же выборке

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-12-12:54:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpnz8nv0gg/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-12-12:54:22
INFO:tensorflow:Saving dict for global step 1: accuracy = 0.2857143, average_loss = 1.8057231, global_step = 1, loss = 12.640061


In [31]:
print(eval_info)

{'accuracy': 0.2857143, 'average_loss': 1.8057231, 'loss': 12.640061, 'global_step': 1}
